In [72]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [73]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 50000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
ff_dropout = 0.0
attn_dropout = 0.2

In [74]:
torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('./input/tinyshakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [75]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(attn_dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(attn_dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(ff_dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head, res1_dropout):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        self.dropout = nn.Dropout(res1_dropout)
        self.res1_dropout = res1_dropout

    # def forward(self, x):
    #     # apply self attention and dropout
    #     x = self.ln1(x)
    #     y = x + self.dropout(self.sa(x))
    #     # apply feedforward and dropout
    #     y = self.ln2(x)
    #     x = y + self.dropout(self.ffwd(y))
    #     return x
    
    # def forward(self, x):
    #     # apply self attention and dropout
    #     y = self.dropout(x) + self.sa(self.ln1(x))
    #     # apply feedforward and dropout
    #     z = self.dropout(y) + self.ffwd(self.ln2(y))
    #     return z
    

    def forward(self, x):
        # apply self attention and dropout
        x = self.ln1(x)
        y = self.dropout(x) * (1 - self.res1_dropout) + self.sa(x)
        # apply feedforward and dropout
        y = self.ln2(y)
        z = self.dropout(y) * (1 - self.res1_dropout) + self.ffwd(y)
        return z


    # def forward(self, x):
    #     # apply self attention and dropout
    #     y = x + self.dropout(self.sa(self.ln1(x)))
    #     # apply feedforward and dropout
    #     z = y + self.dropout(self.ffwd(self.ln2(y)))
    #     return z

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, res1_dropout=0.1):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head, res1_dropout=res1_dropout) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx





In [76]:
import json
import os
import time

res1_dropout_values = [0.0, 0.025, 0.2, 0.5]

for dropout in res1_dropout_values:
    print(f"Training model with dropout={dropout}")

    # Start timing the training process
    start_time = time.time()
    
    # Instantiate the model with the current dropout value
    model = BigramLanguageModel(res1_dropout=dropout).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    
    # Lists to store the loss values for the current dropout experiment
    train_losses = []
    val_losses = []

    for iter in range(max_iters):
        # Evaluate the loss on training and validation sets at certain intervals
        if iter % eval_interval == 0 or iter == max_iters - 1:
            losses = estimate_loss()  # Ensure this function uses the model from the current loop
            train_losses.append(losses['train'].item())
            val_losses.append(losses['val'].item())
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        
        # Sample a batch of data
        xb, yb = get_batch('train')
        
        # Calculate the loss
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    # End timing the training process
    end_time = time.time()
    training_time = end_time - start_time  # Calculate the total training time

    # Results for losses
    loss_results = {
        "train_losses": train_losses,
        "val_losses": val_losses,
    }

    # Result for training time
    time_result = {
        "training_time": training_time,
    }

    # Define the folder path
    folder_path = "./loss_time/res1"
    
    # Ensure the directory exists
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # Filename and path for losses
    loss_filename = f'res1_dropout_{dropout}_losses.json'
    loss_file_path = os.path.join(folder_path, loss_filename)
    
    # Write the loss results to the file
    with open(loss_file_path, 'w') as f:
        json.dump(loss_results, f)
    
    print(f"Loss results for res1 dropout={dropout} saved to {loss_file_path}")
    
    # Filename and path for training time
    time_filename = f'res1_dropout_{dropout}_training_time.json'
    time_file_path = os.path.join(folder_path, time_filename)
    
    # Write the training time to the file
    with open(time_file_path, 'w') as f:
        json.dump(time_result, f)
    
    print(f"Training time for res1 dropout={dropout} saved to {time_file_path}")

Training model with dropout=0.0
step 0: train loss 4.4283, val loss 4.4185
step 100: train loss 2.6480, val loss 2.6604
step 200: train loss 2.5184, val loss 2.5105
step 300: train loss 2.4420, val loss 2.4551
step 400: train loss 2.3771, val loss 2.3806
step 500: train loss 2.3267, val loss 2.3447
step 600: train loss 2.2805, val loss 2.2843
step 700: train loss 2.2431, val loss 2.2532
step 800: train loss 2.2048, val loss 2.2230
step 900: train loss 2.1710, val loss 2.1879
step 1000: train loss 2.1487, val loss 2.1655
step 1100: train loss 2.1182, val loss 2.1531
step 1200: train loss 2.0903, val loss 2.1183
step 1300: train loss 2.0816, val loss 2.1041
step 1400: train loss 2.0496, val loss 2.0769
step 1500: train loss 2.0305, val loss 2.0733
step 1600: train loss 2.0116, val loss 2.0747
step 1700: train loss 2.0039, val loss 2.0547
step 1800: train loss 1.9751, val loss 2.0423
step 1900: train loss 1.9638, val loss 2.0170
step 2000: train loss 1.9465, val loss 2.0311
step 2100: tra

In [77]:
import json
import matplotlib.pyplot as plt
import os

# Specify the folder path where the loss JSON files are stored
folder_path = "./loss_time/res1"
# Specify the path to save the visualization result
visualization_save_path = "./visualization/res1_loss_curves.png"

# Use a pattern or a specific part of the file name to identify loss files
# Assuming loss files contain '_losses' in their filenames
loss_files = [f for f in os.listdir(folder_path) if f.endswith('_losses.json')]

# Visualize the loss curves for each dropout configuration
plt.figure(figsize=(10, 6))

for file in loss_files:
    file_path = os.path.join(folder_path, file)
    
    # Read the loss values from the JSON file
    with open(file_path, 'r') as f:
        losses = json.load(f)
    
    # Plot the validation loss curves
    # Extracting the dropout value from the filename for labeling
    dropout_label = file.split('_')[2]  # Assumes format 'attn_dropout_{value}_losses.json'
    plt.plot(losses['val_losses'], label=f"Validation Loss Dropout {dropout_label}")

plt.title("Loss Curves for Different Attention Dropout Configurations")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)

# Save the image
plt.savefig(visualization_save_path)
plt.close()

print(f"Visualization saved to {visualization_save_path}")




Visualization saved to ./visualization/res1_loss_curves.png
